<a href="https://colab.research.google.com/github/GMayumi/FreeCodeCamp/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# copy and verify
df = dataset.copy()
df.info()

In [ ]:
# Convert categorical data to numbers
print("sex: ", df["sex"].unique())
print("smoker: ", df["smoker"].unique())
print("region: ", df["region"].unique())

print(pd.factorize(df["sex"]))

df["sex"] = pd.factorize(df["sex"])[0]
df["region"] = pd.factorize(df["region"])[0]
df["smoker"] = pd.factorize(df["smoker"])[0]


In [ ]:
# Use 80% of the data as the train_dataset and 20% of the data as the test_dataset
test_dataset = df.sample(frac=0.2)
train_dataset = df[~df.isin(test_dataset)].dropna()

print(df.shape, test_dataset.shape, train_dataset.shape)

In [ ]:
# pop off the "expenses" column from these datasets to create new datasets called train_labels and test_labels
train_labels = train_dataset.pop("expenses")
test_labels = test_dataset.pop("expenses")

print(train_labels.head())
print(train_dataset.head())

In [ ]:
# Create a model
normalizer = layers.Normalization()
normalizer.adapt(np.array(train_dataset))

model = tf.keras.Sequential([
    normalizer,
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1)
])

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mae',
    metrics=['mae', 'mse']
)

model.build()
model.summary()

In [ ]:
# Train the model with the train_dataset
history = model.fit(
    train_dataset,
    train_labels,
    epochs=100
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
